# 2.0 - Data Quality

In [ ]:
# import
import os
import json
from openai import OpenAI
MODEL = os.getenv('MODEL')
API_KEY = os.getenv('CHATGPT_TOKEN')
iter = 13300 # to do

#failed: 27100, 27000, 26900, 26700, 26600, 26500
#failed: 26400, 26300, 26000, 25900, 25800, 25700, 25500, 25400, 25200, 25100, 25000, 24900, 24800, 24700, 24600
#failed: 24500, 24400, 24200, 24100, 24000, 23900, 23800, 23700, 23600, 23500, 23400, 23300, 23200, 23100, 23000,
#failed: 22900, 22800, 22700, 22600, 22500, 22400, 22300, 22200, 21900, 21800, 21700, 21600, 21500

In [113]:
# setup
client = OpenAI(api_key=API_KEY) #initialize the OpenAI client

In [ ]:
# check currently active batch process
#[el for el in client.batches.list() if el.status == 'in_progress']

---

##### OpenAI documentation: https://platform.openai.com/docs/guides/batch?lang=python

In [114]:
#define batch input file
batch_input_file = client.files.create(
    file = open(f'../data/processed/requests/requests_{iter}.jsonl', 'rb'),
    purpose = 'batch'
)

In [115]:
batch_input_file_id = batch_input_file.id
# create the batch process
batch_object = client.batches.create(
    input_file_id = batch_input_file_id,
    endpoint = '/v1/chat/completions',
    completion_window = '24h',
    metadata = {
        'description': 'nightly eval job'
    }
)
batch_object_id = batch_object.id
with open(f'../.env', 'a') as f:
    f.write(f"BATCH_PROCESS_ID_{iter} = '{batch_object.id}'\n")
    f.close()
#print(f"BATCH_PROCESS_ID_{iter} = '{batch_object.id}'")

---

batch_object_id = os.getenv(f'BATCH_PROCESS_ID_{iter}')
batch_object_id = batch_object.id if batch_object else batch_object_id
batch_object_id

In [119]:
# retrieve data from batch process
batch = client.batches.retrieve(batch_object.id)
print(f'Batch Object id: {batch.id}')
print(f'Status: {batch.status} - {batch.request_counts}')
print(f'Batch file: {batch.output_file_id}')

Batch Object id: batch_67c87bf362348190b127a1632360eaa5
Status: completed - BatchRequestCounts(completed=100, failed=0, total=100)
Batch file: file-4cPnSZdYPxyPxwhqW9rseE


In [120]:
# get the output file
file_response = client.files.content(batch.output_file_id)

In [121]:
# save results to a jsonl file
json_list = list()
with open(f'../data/processed/responses/responses_{iter}.jsonl', 'w') as f:
    for response in [x for x in file_response.text.split('\n') if x]:
        json.dump(json.loads(response), f)
        json_file = json.loads(response)
        current_json = {
            'text' : json_file['response']['body']['choices'][0]['message']['content'],
            'doc_id' : json_file['custom_id']
        }
        json_list.append(current_json.copy())
        f.write('\n')